In [28]:
import ROOT
import numpy as np

# Parameters
n_bins = 10
x_min = 0
x_max = 10

# TH1D with statistical uncertainties only
hist_stat = ROOT.TH1D("hist_stat", "Exponential Spectrum;GeV;Dummy cross-section [mb]", n_bins, x_min, x_max)
hist_halved = ROOT.TH1D("hist_halved", "Exponential Spectrum;GeV;Dummy halved cross-section [mb]", n_bins, x_min, x_max)

# Prepare arrays for TGraphAsymmErrors
x_vals, y_vals, ex_low, ex_high, ey_low, ey_high = [], [], [], [], [], []

# Fill both hist_stat and TGraphAsymmErrors with identical points
for i in range(1, n_bins + 1):
    x_center = hist_stat.GetBinCenter(i)
    y_value = np.exp(-0.5 * x_center)*100  # Shared y value
    y_value_err = np.sqrt(y_value)
    y_value_err = y_value_err / 100
    y_value = y_value/100

    # Fill histogram
    hist_stat.SetBinContent(i, y_value)
    hist_halved.SetBinContent(i, y_value/2)
    hist_stat.SetBinError(i, y_value_err)

    # Fill graph data (same x, y)
    x_vals.append(x_center)
    y_vals.append(y_value)
    ex_low.append(0.5)
    ex_high.append(0.5)
    ey_low.append(0.1 * y_value *i/2)
    ey_high.append(0.3 * y_value*i/2)

# Create TGraphAsymmErrors
graph = ROOT.TGraphAsymmErrors(n_bins,
    np.array(x_vals, dtype='float64'),
    np.array(y_vals, dtype='float64'),
    np.array(ex_low, dtype='float64'),
    np.array(ex_high, dtype='float64'),
    np.array(ey_low, dtype='float64'),
    np.array(ey_high, dtype='float64'))
graph.SetName("graph_asym")
graph.SetTitle("Exponential Spectrum with Systematics;GeV;Cross-section [mb]")
graph.SetMarkerStyle(21)
graph.SetMarkerSize(0)
graph.SetLineColor(ROOT.kRed)
graph.SetMarkerColor(ROOT.kRed)
graph.SetFillColorAlpha(ROOT.kBlue, 0.35)

# Create a random symmetric correlation matrix as TH2D
np.random.seed(123)
R = np.random.rand(n_bins, n_bins) * 2 - 1  # Values in [-1, 1]
R = (R + R.T) / 2
np.fill_diagonal(R, 1.0)

corr_matrix = ROOT.TH2D("corr_matrix", "Random Correlation Matrix;Bin Index;Bin Index",
                        n_bins, 0.5, n_bins + 0.5, n_bins, 0.5, n_bins + 0.5)

for i in range(n_bins):
    for j in range(n_bins):
        corr_matrix.SetBinContent(i + 1, j + 1, R[i, j])

hist_stat.GetYaxis().SetRangeUser(1E-3,2)

hist_halved.SetMarkerStyle(21)
        
canvas = ROOT.TCanvas("c","c",1200,1200)
canvas.SetLogy()
hist_stat.Draw("E1")
graph.Draw("P2 same")
hist_halved.Draw("P same")
canvas.Draw()
canvas.SaveAs("spectrum.png")


canvas1 = ROOT.TCanvas("c1","c1",1200,1200)
corr_matrix.Draw()
canvas1.Draw()
canvas1.SaveAs("corr_matrix.png")



Warning in <TROOT::Append>: Replacing existing TH1: hist_stat (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hist_halved (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: corr_matrix (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: png file spectrum.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Info in <TCanvas::Print>: png file corr_matrix.png has been created


In [27]:
# Save everything to a ROOT file
output_file = ROOT.TFile("spectrum_output.root", "RECREATE")
hist_stat.Write()
hist_halved.Write()
graph.Write()
corr_matrix.Write()
output_file.Close()